# Milestone 4: Analysis and Visualisation 

In [1]:
# Preamble - imports 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sbn 

In [2]:
# finance_df = pd.read_csv("df_after_Task_6.csv") # All should be cleaned now, dates parsed, etc. 
# type(finance_df) # confirming it's a pd.df :)  
# I've commented out the df above since it got rid of all my useful columns- I will use the data 'cleaned' up to but not including the removal of highly correlated columns instead. 
finance_df = pd.read_csv("df_end_of_Task_5.csv")

**Aside:** Upon trying to find suitable columns to 'summarise currently what percentage of the loans are recovered against hte investor funding and the total amount funded,' I realised that data cleaning task number 6 removed all the potentially appropriate columns for that task, namely: 
- **out_prncp**: Remaining outstanding principal for total amount funded
- **out_prncp_inv**: Remaining outstanding principal for portion of total amount funded by investors
- **total_payment**: Payments received to date for total amount funded

I am shocked by the pedagogical blundering that would lead to such an oversight- students don't know where they're going or what sort of analysis will be asked of them, so they don't know how to properly clean the data without utterly destroying its utility, and the casually wirtten data 'cleaning' prompts encourage a cavalier attitude about maintaining data. 

This is why I'm importing the csv from the end of task 5 instead of task 6. 

In [ ]:
# print(finance_df.dtypes) # plopped into loan_data_dict.md for further reference. 

## Task 1: Current state of loans 
1. Summarise currently what percentage of the loans are recovered against the investor funding and the total amount funded. 
2. Visualise the results of the above. 
3. Visualise what percentage of the total amount would be recovered up to 6 months in the future. 

For the first prompt, I'll use the following columns: 
- **out_prncp_inv**: Remaining outstanding principal for portion of total amount funded by investors

In [1]:
out_prncp_inv = finance_df["out_prncp_inv"]

out_prncp_inv.nlargest(10)

out_prncp_inv.nsmallest(10)

out_prncp_inv.mean()

out_prncp_inv.median() 


#out_prncp_inv.head(20)


NameError: name 'finance_df' is not defined